In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

################################################################################################

def create_dataframe(doc_tf, doc_targets):
    docs = []
    for i, bow in enumerate(doc_tf):
        d = {'doc_id': i, 'bow': bow, 'label': doc_targets[i]}
        docs.append(d)
    return pd.DataFrame.from_dict(docs)

train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes')) 
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

################################################################################################
count_vect = CountVectorizer(stop_words='english', max_features=10000, max_df=0.8, min_df=3)
train_tf = count_vect.fit_transform(train.data)
test_tf = count_vect.transform(test.data)

train_df = create_dataframe(train_tf, train.target)
test_df = create_dataframe(test_tf, test.target)

def get_doc_length(doc_bow):
    return doc_bow.sum()

# remove an empty document
train_df = train_df[train_df.bow.apply(get_doc_length) > 0]
test_df = test_df[test_df.bow.apply(get_doc_length) > 0]

# split test and cv
num_train = len(train_df)
num_test = len(test_df) // 2
num_cv = len(test_df) - num_test

test_df = shuffle(test_df)
cv_df = test_df.iloc[:num_cv]
test_df = test_df.iloc[num_cv:]

# set doc_id as an index
train_df.set_index('doc_id', inplace=True)
test_df.set_index('doc_id', inplace=True)
cv_df.set_index('doc_id', inplace=True)

In [2]:
# save the dataframes
train_df.to_pickle('../dataset/ng20/train.tf.df.pkl')
test_df.to_pickle('../dataset/ng20/test.tf.df.pkl')
cv_df.to_pickle('../dataset/ng20/cv.tf.df.pkl')

# save vocab
with open('../dataset/ng20/vocab.pkl', 'wb') as handle:
    pickle.dump(count_vect.vocabulary_, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# TFIDF format
from scipy.sparse import csr_matrix, vstack

# compute DocFrequency
doc_freq = np.zeros(train_df.iloc[0].bow.shape[1])
for index, row in train_df.iterrows():
    bow = (row.bow.toarray().squeeze() > 0).astype(np.float)
    doc_freq += bow
    
# compute inverse document frequency
N = len(train_df)
idf = np.log(1. + N / doc_freq)

def create_tfidf_matrix(doc_tf_df, docidf):
    # create TFIDF
    doc_tfidf = []
    for index, row in doc_tf_df.iterrows():
        bow = (row.bow.toarray().squeeze()).astype(np.float)
        tfidf = csr_matrix(np.log1p(bow) * docidf)
        doc_tfidf.append(tfidf)
    return vstack(doc_tfidf)

train_tfidf_df = create_dataframe(create_tfidf_matrix(train_df, idf), list(train_df.label))
test_tfidf_df = create_dataframe(create_tfidf_matrix(test_df, idf), list(test_df.label))
cv_tfidf_df = create_dataframe(create_tfidf_matrix(cv_df, idf), list(cv_df.label))

# save the dataframes
train_tfidf_df.set_index('doc_id', inplace=True)
test_tfidf_df.set_index('doc_id', inplace=True)
cv_tfidf_df.set_index('doc_id', inplace=True)

train_tfidf_df.to_pickle('../dataset/ng20/train.tfidf.df.pkl')
test_tfidf_df.to_pickle('../dataset/ng20/test.tfidf.df.pkl')
cv_tfidf_df.to_pickle('../dataset/ng20/cv.tfidf.df.pkl')

In [8]:
# Binary format
from scipy.sparse import csr_matrix, vstack

def create_bin_matrix(doc_tf_df):
    # create TFIDF
    doc_bin = []
    for index, row in doc_tf_df.iterrows():
        bow = (row.bow.toarray().squeeze() > 0).astype(np.float)
        bow = csr_matrix(bow)
        doc_bin.append(bow)
    return vstack(doc_bin)

train_bin_df = create_dataframe(create_bin_matrix(train_df), list(train_df.label))
test_bin_df = create_dataframe(create_bin_matrix(test_df), list(test_df.label))
cv_bin_df = create_dataframe(create_bin_matrix(cv_df), list(cv_df.label))

# save the dataframes
train_bin_df.to_pickle('../dataset/ng20/train.bin.df.pkl')
test_bin_df.to_pickle('../dataset/ng20/test.bin.df.pkl')
cv_bin_df.to_pickle('../dataset/ng20/cv.bin.df.pkl')

In [ ]:
# import os
# from os.path import join
# import numpy as np
# import pandas as pd
# import pickle
# import torch
# from torch.utils.data import Dataset

# class Newsgroups20Dataset(Dataset):
#     """Newsgroups20 dataset."""

#     def __init__(self, data_dir, download=False, subset='train', bow_format='tf'):
#         """
#         Args:
#             data_dir (string): Directory for loading and saving train, test, and cv dataframes.
#             download (boolean): Download newsgroups20 dataset from sklearn if necessary.
#             subset (string): Specify subset of the datasets. The choices are: train, test, cv.
#             bow_format (string): A weight scheme of a bag-of-words document. The choices are:
#                 tf (term frequency), tfidf (term freq with inverse document frequency), bm25.
#         """
#         self.data_dir = data_dir
#         self.subset = subset
#         self.bow_format = bow_format
#         self.df = self.load_df('{}.{}.df.pkl'.format(subset, bow_format))
        
#     def load_df(self, df_file):
#         df_file = os.path.join(self.data_dir, df_file)
#         return pd.read_pickle(df_file)
        
#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         doc_bow = self.df.iloc[idx].bow
#         doc_bow = torch.from_numpy(doc_bow.toarray().squeeze().astype(np.float32))
#         label = self.df.iloc[idx].label
#         return (doc_bow, label)

In [ ]:
train_set = Newsgroups20Dataset('../dataset/ng20', subset='train', download=True, bow_format='tf')
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=64, shuffle=True)

In [ ]:
for xb, yb in train_loader:
    break

In [ ]:
xb.size()

In [ ]:
yb